In [4]:
# Import necessary libraries
import os 
from typing import Any, Dict

import google.generativeai as genai 
import requests
from dotenv import load_dotenv

print("We are up and running")

We are up and running


Ensure that you created a Gemini API Key [Here](https://aistudio.google.com/apikey) and added it to the .env file before executing the next command

In [11]:
# load environment variables 
load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')
if not api_key or api_key.startswith('ADD YOUR'):
    raise ValueError('GEMINI_API_KEY not found in .env file')

# Configure Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.0-flash')

print(f'Gemini model loaded successfully: {model.model_name}')

Gemini model loaded successfully: models/gemini-2.0-flash


In [26]:
PROMPT = 'What is the symbol of Solana?'
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(response.text)

The symbol of Solana is **SOL**.



In [13]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I cannot provide you with real-time, up-to-the-minute price data for Solana (SOL). Cryptocurrency prices are highly volatile and change constantly.\n\nHowever, here's how you can find the current price of Solana:\n\n*   **Reputable Cryptocurrency Exchanges:** Check major exchanges like Coinbase, Binance, Kraken, Gemini, or KuCoin. These exchanges usually provide real-time price charts and trading information.\n*   **Cryptocurrency Tracking Websites:** Use websites like CoinMarketCap, CoinGecko, or Crypto.com. These sites aggregate price data from various exchanges and provide charts, market capitalization, and other relevant information.\n*   **Financial News Websites:** Some financial news outlets (e.g., Bloomberg, Reuters, CNBC) provide cryptocurrency pric

In [14]:
url = f'https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT'
response = requests.get(url)
data = response.json()
print(data)

{'symbol': 'BTCUSDT', 'price': '106805.66000000'}


In [16]:
# Define the function
def get_crypto_price(symbol: str) -> Dict[str, Any]:
    """
    Get the current price of a cryptocurrency from BINANCE API
    """

    url = f'https://api.binance.com/api/v3/ticker/price?symbol={symbol}'
    response = requests.get(url)
    data = response.json()
    return float(data['price'])


In [18]:
price = get_crypto_price('BTCUSDT')
print(f'BTC price in USDT: {price}')

BTC price in USDT: 106951.06


In [35]:
tools = [
    {
        'function_declarations': [
            {
                'name': 'get_crypto_price',
                'description': 'Get cryptocurrency price in USDT from Binance',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'symbol': {
                            'type': 'string',
                            'description': 'The cryptocurrency trading pair symbol (e.g, BTCUSDT, ETHUSDT). \
                                            The symbol for Bitcoin is BTCUSDT. \
                                            The symbol for Ethereum is ETHUSDT. \ '
                        }
                    },
                    'required': ['symbol']
                }
            }
        ]
    }
]

In [36]:
PROMPT = 'What is the current price of Ethereum?'
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "ETHUSDT"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.006602738983929157
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 62,
        "candidates_token_count": 8,
        "total_token_count": 70
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [37]:
price = get_crypto_price('ETHUSDT')

In [38]:
final_response = chat.send_message(str(price))
print(final_response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The current price of Ethereum is $2644.72.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.07664043456315994
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 23,
        "candidates_token_count": 16,
        "total_token_count": 39
      },
      "model_version": "gemini-2.0-flash"
    }),
)
